## 에코

In [ ]:
import os
import json
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import nltk
nltk.download('punkt')

# 비디오 폴더 경로 설정
VIDEO_FOLDER = "E:/dog_videos/"

# 문장 임베딩을 위한 모델 로드
sentence_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

def compare_results(echo_result, gpt4o_result):
    # 텍스트 임베딩 생성
    echo_embedding = sentence_model.encode([echo_result])
    gpt4o_embedding = sentence_model.encode([gpt4o_result])

    # 코사인 유사도 계산
    similarity = cosine_similarity(echo_embedding, gpt4o_embedding)[0][0]

    # 키워드 추출 및 비교
    echo_keywords = set(echo_result.lower().split())
    gpt4o_keywords = set(gpt4o_result.lower().split())

    common_keywords = echo_keywords.intersection(gpt4o_keywords)
    unique_echo = echo_keywords - gpt4o_keywords
    unique_gpt4o = gpt4o_keywords - echo_keywords

    return {
        "similarity": similarity,
        "common_keywords": list(common_keywords),
        "unique_echo": list(unique_echo),
        "unique_gpt4o": list(unique_gpt4o)
    }

def main():
    results = []

    for video_file in os.listdir(VIDEO_FOLDER):
        if video_file.endswith(('.mp4', '.avi', '.mov')):  # 비디오 파일 확장자
            video_path = os.path.join(VIDEO_FOLDER, video_file)
            print(f"처리 중인 비디오: {video_file}")

            # 기존의 분석 코드 (YOLO, LSTM, LLM 등)
            # ...

            # Echo Demo 분석 결과
            echo_result = generate_comprehensive_analysis(yolo_text, lstm_text, summary_text, rag_text)

            # GPT-4o-mini 분석 결과
            gpt4o_prompt = f"""
            강아지 영상 분석 결과:
            1. YOLO 모델 (품종 탐지): {yolo_text}
            2. LSTM 모델 (행동 분석): {lstm_text}
            3. 비디오 요약: {summary_text}

            위 정보를 바탕으로 강아지의 상태, 행동, 감정에 대해 종합적으로 분석해주세요.
            """
            gpt4o_response = client.chat.completions.create(
                model="gpt-4o-mini-2024-07-18",
                messages=[
                    {"role": "system", "content": "당신은 강아지 행동 분석 전문가입니다."},
                    {"role": "user", "content": gpt4o_prompt}
                ],
                max_tokens=150
            )
            gpt4o_result = gpt4o_response.choices[0].message.content

            # 결과 비교
            comparison = compare_results(echo_result, gpt4o_result)
            results.append({
                "video": video_file,
                "echo_result": echo_result,
                "gpt4o_result": gpt4o_result,
                "comparison": comparison
            })

            print(f"비디오 {video_file} 처리 완료")
            print("유사도:", comparison["similarity"])
            print("공통 키워드:", ", ".join(comparison["common_keywords"][:10]))  # 처음 10개만 출력
            print("Echo Demo 고유 키워드:", ", ".join(comparison["unique_echo"][:10]))
            print("GPT-4o-mini 고유 키워드:", ", ".join(comparison["unique_gpt4o"][:10]))
            print("\n" + "="*50 + "\n")

    # 전체 결과 요약
    avg_similarity = sum(r['comparison']['similarity'] for r in results) / len(results)
    total_common_keywords = set()
    total_unique_echo = set()
    total_unique_gpt4o = set()

    for r in results:
        total_common_keywords.update(r['comparison']['common_keywords'])
        total_unique_echo.update(r['comparison']['unique_echo'])
        total_unique_gpt4o.update(r['comparison']['unique_gpt4o'])

    print("=== 전체 평가 결과 ===")
    print(f"평균 유사도: {avg_similarity:.4f}")
    print(f"총 공통 키워드 수: {len(total_common_keywords)}")
    print(f"Echo Demo 총 고유 키워드 수: {len(total_unique_echo)}")
    print(f"GPT-4o-mini 총 고유 키워드 수: {len(total_unique_gpt4o)}")

    # 결과를 JSON 파일로 저장
    with open('analysis_results.json', 'w', encoding='utf-8') as f:
        json.dump(results, f, ensure_ascii=False, indent=2)

if __name__ == "__main__":
    main()

## GPT-4o-mini

In [1]:
%pip install openai

Note: you may need to restart the kernel to use updated packages.


In [12]:
import json
from openai import OpenAI
from collections import Counter

client = OpenAI(api_key=OPENAI_API_KEY)

def process_yolo_results(results):
    lstm_keypoint_sequence = []
    skeleton_sequence = []
    breed_counter = Counter()

    for r in results:
        if r.keypoints is not None and len(r.keypoints) > 0:
            yolo_keypoints = r.keypoints[0].cpu().numpy()
            lstm_keypoints = convert_yolo_to_lstm(yolo_keypoints)
            lstm_keypoint_sequence.append(lstm_keypoints)
            skeleton = create_skeleton(lstm_keypoints)
            skeleton_sequence.append(skeleton)

            # YOLO 결과에서 품종 정보 추출 및 카운트
            boxes = r.boxes
            for box in boxes:
                cls = int(box.cls[0])
                class_name = r.names[cls]
                breed_counter[class_name] += 1  # 품종 등장 횟수 증가

    return np.array(lstm_keypoint_sequence), np.array(skeleton_sequence), breed_counter


def analyze_keypoints_with_llm(json_data):
    # 키포인트 정보만 추출
    annotations = json_data['annotations']
    
    # 키포인트 데이터 문자열 생성 (전체 프레임)
    keypoints_str = json.dumps(annotations, indent=2, ensure_ascii=False)
    
    # LLM에 전달할 프롬프트 생성
    prompt = f"""
    다음은 고양이의 키포인트 데이터입니다. 이 데이터를 바탕으로 고양이의 행동을 분석해주세요. 

    {keypoints_str}

    이 데이터를 바탕으로 고양이의 행동을 분석해주세요. 
    다음 사항들을 고려하여 분석해 주세요:
    1. 고양이의 자세
    2. 움직임의 패턴
    3. 머리, 등, 꼬리의 위치
    4. 시간에 따른 키포인트의 변화

    분석 결과에는 다음 내용을 포함해 주세요:
    1. 고양이가 취하고 있는 주요 행동
    2. 고양이의 감정 상태 추정
    3. 고통이나 질병의 징후가 있는지 여부
    4. 비정상적인 행동이 있는지 여부

    핵심만 짧게 요약해주세요.
    출력은 아래 형태로 제공해주세요.
    [품종]:[품종 추정, 모르면 '알수없음'],
    [행동]:[주요 행동을 한 문장으로 설명],
    [감정]:[고양이의 감정 상태를 한 문장으로 설명],
    [질병/통증]:[질병이나 통증이 있는지 여부를 한 문장으로 설명],
    [비정상행동]:[비정상적인 행동이 있는지 여부를 한 문장으로 설명]
    """

    # LLM API 호출  
    response = client.chat.completions.create(
        model="gpt-4o-mini-2024-07-18",
        messages=[
            {"role": "system", "content": "당신은 동물 행동 분석 전문가입니다. 키포인트 데이터를 바탕으로 고양이의 행동을 분석하고 감정 상태, 질병과 통증여부, 비정상 행동 여부를 체크할 수 있습니다."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=1000
    )

    return response.choices[0].message.content

def compare_analysis_with_metadata(analysis_result, json_data):
    print("LLM 분석 결과:")
    print(analysis_result)
    print("\n실제 메타데이터:")
    
    metadata = json_data.get('metadata', {})
    if metadata:
        print(json.dumps(metadata, indent=2, ensure_ascii=False))
    else:
        print("메타데이터를 찾을 수 없습니다.")
    
    print("\n비교 분석:")
    # LLM 분석 결과와 메타데이터 비교
    if metadata:
        inspect = metadata.get('inspect', {})
        print(f"실제 행동: {inspect.get('action', '정보 없음')}")
        print(f"실제 감정: {inspect.get('emotion', '정보 없음')}")
        print(f"통증/질병 여부: {inspect.get('painDisease', '정보 없음')}")
        print(f"비정상 행동 여부: {inspect.get('abnormalAction', '정보 없음')}")
        
        # 여기에 LLM 분석 결과와 메타데이터를 비교하는 추가 로직을 구현할 수 있습니다.
    else:
        print("메타데이터가 없어 비교 분석을 수행할 수 없습니다.")

# JSON 파일 로드 및 분석 실행 부분
with open('E:/LSTN_test/data/ARCH/20201028_cat-arch-000156.mp4.json', 'r', encoding='utf-8') as f:
    json_data = json.load(f)

analysis_result = analyze_keypoints_with_llm(json_data)

# 분석 결과와 메타데이터 비교
compare_analysis_with_metadata(analysis_result, json_data)

LLM 분석 결과:
[품종]: 알수없음,  
[행동]: 고양이는 주로 앉아 있거나 엎드린 자세로 안정된 상태를 유지하며 주위를 살피고 있다.  
[감정]: 고양이는 긴장을 풀고 편안한 상태로 보이며, 주위를 탐색하는 모습에서 호기심이 느껴진다.  
[질병/통증]: 전반적으로 움직임이 부드럽고 자연스럽기 때문에 질병이나 통증의 징후는 보이지 않는다.  
[비정상행동]: 비정상적인 행동은 발견되지 않으며, 정상적인 고양이 행동을 보인다.  

실제 메타데이터:
{
  "seq": 156,
  "species": "CAT",
  "action": "허리를 아치로 세움",
  "location": "실내",
  "height": 1440,
  "width": 1440,
  "duration": 17.408,
  "animal": {
    "breed": "코리안 숏헤어",
    "gender": "FEMALE",
    "age": 1,
    "neuter": "Y"
  },
  "owner": {
    "pain": "N",
    "disease": "N",
    "emotion": "화남/불쾌",
    "situation": "낯선 장소에 있거나 낯선 소리가 날 때",
    "animalCount": 1
  },
  "inspect": {
    "action": "허리를 아치로 세움",
    "painDisease": "N",
    "abnormalAction": "N",
    "emotion": "화남/불쾌"
  }
}

비교 분석:
실제 행동: 허리를 아치로 세움
실제 감정: 화남/불쾌
통증/질병 여부: N
비정상 행동 여부: N


In [ ]:
import os
import json
import tkinter as tk
from tkinter import filedialog
from openai import OpenAI
from ultralytics import YOLO
import cv2
import numpy as np

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
yolo_model = YOLO("E:/1006/model/yolo_model.pt")
conf_thresh = 0.7

def process_yolo_keypoints(results):
    keypoints_sequence = []
    for r in results:
        if r.keypoints is not None:
            frame_keypoints = r.keypoints.xy[0].cpu().numpy().tolist()
            keypoints_sequence.append(frame_keypoints)
    return keypoints_sequence

def analyze_keypoints_with_llm(keypoints_sequence):
    keypoints_str = json.dumps(keypoints_sequence, indent=2)
    
    prompt = f"""
    다음은 개의 키포인트 데이터 시퀀스입니다. 각 프레임마다 24개의 키포인트가 있으며, 순서는 다음과 같습니다:
    0: Front Left Paw, 1: Front Left Knee, 2: Front Left Elbow, 3: Rear Left Paw, 4: Rear Left Knee,
    5: Rear Left Elbow, 6: Front Right Paw, 7: Front Right Knee, 8: Front Right Elbow, 9: Rear Right Paw,
    10: Rear Right Knee, 11: Rear Right Elbow, 12: Tail Start, 13: Tail End, 14: Left Ear Base,
    15: Right Ear Base, 16: Nose, 17: Chin, 18: Left Ear Tip, 19: Right Ear Tip, 20: Left Eye,
    21: Right Eye, 22: Withers, 23: Throat

    키포인트 데이터:
    {keypoints_str}

    이 데이터를 바탕으로 개의 행동을 분석해주세요. 다음 사항들을 고려하여 분석해 주세요:
    1. 개의 자세와 움직임 패턴
    2. 머리, 꼬리, 귀의 위치와 움직임
    3. 시간에 따른 키포인트의 변화

    분석 결과에는 다음 내용을 포함해 주세요:
    1. 개가 취하고 있는 주요 행동
    2. 개의 감정 상태 추정
    3. 고통이나 질병의 징후가 있는지 여부
    4. 비정상적인 행동이 있는지 여부

    출력은 아래 형태로 제공해주세요:
    [행동]:[주요 행동을 한 문장으로 설명],
    [감정]:[개의 감정 상태를 한 문장으로 설명],
    [질병/통증]:[질병이나 통증이 있는지 여부를 한 문장으로 설명],
    [비정상행동]:[비정상적인 행동이 있는지 여부를 한 문장으로 설명]
    """

    response = client.chat.completions.create(
        model="gpt-4o-mini-2024-07-18",
        messages=[
            {"role": "system", "content": "당신은 동물 행동 분석 전문가입니다. 키포인트 데이터를 바탕으로 개의 행동을 정확하게 분석할 수 있습니다."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=1000
    )

    return response.choices[0].message.content

def analyze_video():
    file_path = filedialog.askopenfilename(filetypes=[("Video files", "*.mp4;*.avi;*.mov")])
    if file_path:
        # YOLO 모델 예측 및 키포인트 추출
        results = yolo_model.predict(source=file_path, save=False, conf=conf_thresh, stream=True, verbose=False)
        keypoints_sequence = process_yolo_keypoints(results)

        # LLM 분석
        llm_analysis = analyze_keypoints_with_llm(keypoints_sequence)
        
        # 결과 표시
        result_window = tk.Toplevel(root)
        result_window.title("분석 결과")
        result_text = tk.Text(result_window, wrap=tk.WORD, width=60, height=20)
        result_text.insert(tk.END, llm_analysis)
        result_text.pack(padx=10, pady=10)

# GUI 생성
root = tk.Tk()
root.title("개 행동 분석기")

upload_button = tk.Button(root, text="비디오 업로드 및 분석", command=analyze_video)
upload_button.pack(pady=20)

root.mainloop()

In [ ]:
import os
import json
from sklearn.metrics import accuracy_score, f1_score
from nltk.translate.bleu_score import sentence_bleu
import nltk
nltk.download('punkt')

# 비디오 폴더와 모범 답안 파일 경로 설정
VIDEO_FOLDER = "E:/dog_videos/"
GROUND_TRUTH_FILE = "E:/dog_videos/ground_truth.json"

def load_ground_truth():
    with open(GROUND_TRUTH_FILE, 'r', encoding='utf-8') as f:
        return json.load(f)

def evaluate_results(ground_truth, echo_result, gpt4o_result):
    # BLEU 점수 계산
    reference = nltk.word_tokenize(ground_truth.lower())
    echo_bleu = sentence_bleu([reference], nltk.word_tokenize(echo_result.lower()))
    gpt4o_bleu = sentence_bleu([reference], nltk.word_tokenize(gpt4o_result.lower()))

    # 간단한 키워드 기반 정확도 계산
    keywords = set(ground_truth.lower().split())
    echo_accuracy = len(set(echo_result.lower().split()) & keywords) / len(keywords)
    gpt4o_accuracy = len(set(gpt4o_result.lower().split()) & keywords) / len(keywords)

    return {
        "echo_bleu": echo_bleu,
        "gpt4o_bleu": gpt4o_bleu,
        "echo_accuracy": echo_accuracy,
        "gpt4o_accuracy": gpt4o_accuracy
    }

def main():
    ground_truths = load_ground_truth()
    results = []

    for video_file in os.listdir(VIDEO_FOLDER):
        if video_file.endswith(('.mp4', '.avi', '.mov')):  # 비디오 파일 확장자
            video_path = os.path.join(VIDEO_FOLDER, video_file)
            print(f"처리 중인 비디오: {video_file}")

            # 기존의 분석 코드 (YOLO, LSTM, LLM 등)
            # ...

            # Echo Demo 분석 결과
            echo_result = generate_comprehensive_analysis(yolo_text, lstm_text, summary_text, rag_text)

            # GPT-4o-mini 분석 결과
            gpt4o_prompt = f"""
            강아지 영상 분석 결과:
            1. YOLO 모델 (품종 탐지): {yolo_text}
            2. LSTM 모델 (행동 분석): {lstm_text}
            3. 비디오 요약: {summary_text}

            위 정보를 바탕으로 강아지의 상태, 행동, 감정에 대해 종합적으로 분석해주세요.
            """
            gpt4o_response = client.chat.completions.create(
                model="gpt-4o-mini-2024-07-18",
                messages=[
                    {"role": "system", "content": "당신은 강아지 행동 분석 전문가입니다."},
                    {"role": "user", "content": gpt4o_prompt}
                ],
                max_tokens=150
            )
            gpt4o_result = gpt4o_response.choices[0].message.content

            # 결과 평가
            ground_truth = ground_truths.get(video_file, "")
            evaluation = evaluate_results(ground_truth, echo_result, gpt4o_result)
            results.append({
                "video": video_file,
                "ground_truth": ground_truth,
                "echo_result": echo_result,
                "gpt4o_result": gpt4o_result,
                "evaluation": evaluation
            })

            print(f"비디오 {video_file} 처리 완료")
            print("평가 결과:", evaluation)
            print("\n" + "="*50 + "\n")

    # 전체 결과 요약
    avg_echo_bleu = sum(r['evaluation']['echo_bleu'] for r in results) / len(results)
    avg_gpt4o_bleu = sum(r['evaluation']['gpt4o_bleu'] for r in results) / len(results)
    avg_echo_accuracy = sum(r['evaluation']['echo_accuracy'] for r in results) / len(results)
    avg_gpt4o_accuracy = sum(r['evaluation']['gpt4o_accuracy'] for r in results) / len(results)

    print("=== 전체 평가 결과 ===")
    print(f"Echo Demo 평균 BLEU 점수: {avg_echo_bleu:.4f}")
    print(f"GPT-4o-mini 평균 BLEU 점수: {avg_gpt4o_bleu:.4f}")
    print(f"Echo Demo 평균 정확도: {avg_echo_accuracy:.4f}")
    print(f"GPT-4o-mini 평균 정확도: {avg_gpt4o_accuracy:.4f}")

if __name__ == "__main__":
    main()